In [137]:
import pandas as pd
import numpy as np

In [138]:
pickup = pd.read_csv("pickup_summary.csv")
pickup = pickup.dropna()
dropoff = pd.read_csv("dropoff_summary.csv")
dropoff = dropoff.dropna()

In [139]:
def filter_station(df, col):
    count = df.groupby([col]).size().to_frame(name = 'count').reset_index()
    count = count[count['count']>100]
    return count[col].unique()


stations = np.intersect1d(filter_station(pickup, 'startstationname'), filter_station(dropoff, 'endstationname'))
dates = pickup['date'].unique()

In [140]:
df = np.transpose([np.tile(stations, len(dates)), np.repeat(dates, len(stations))])
half_day = np.array([0,1])
df = pd.DataFrame(np.tile(df, (2,1)), columns = ['station', 'date'])
df['half_day'] = np.repeat(half_day, len(df)/2)

In [141]:
df = pd.merge(df, pickup[['startstationname','date', 'half_day', 'pickup']],  how='left', left_on=['station','date', 'half_day'], right_on = ['startstationname','date', 'half_day'])
df = pd.merge(df, dropoff[['endstationname','date', 'half_day', 'dropoff']],  how='left', left_on=['station','date', 'half_day'], right_on = ['endstationname','date', 'half_day'])

In [142]:
weather = pd.read_csv("weather.csv")
df = pd.merge(df, weather,  how='left', left_on=['date'], right_on = ['timestamp'])

In [143]:
covid = pd.read_csv("covid.csv")
df = pd.merge(df, covid,  how='left', left_on=['date'], right_on = ['date_of_interest'])

In [147]:
df = df[['station', 'date', 'half_day', 'pickup', 'dropoff', 'AVG_TEMP', 'CASE_COUNT', 'DEATH_COUNT']].fillna(0)

In [149]:
df.to_csv('data.csv')